## U-Net

<img src="https://miro.medium.com/v2/resize:fit:1400/1*qNdglJ1ORP3Gq77MmBLhHQ.png">

U-Net은 기본적인 인코더-디코더 모델이며, **Convolution block**과 **skip connection**을 사용하여 **image segmentation**에 특화된 모델이다.

- 인코더 
    - 일반적인 컨볼루션 신경망(CNN)처럼 2개의 컨볼루션 블록과 1개의 max pooling으로 구성된 단계를 반복
    - Convolution block은 보통 convolution, activation function, normalization으로 이루어져 있음
    - 이미지의 공간적(local feature) 해상도를 줄이면서 (Down sampling) 채널 수를 늘려 특징을 추출하고, 전반적인 맥락 정보 (context information)를 학습

- 디코더
    - 인코더와 대칭적인 구조로, 전치 컨볼루션(Transpose Convolution) 또는 업샘플링(Upsampling)을 통해 특징 맵의 크기를 늘림
    - 이 과정에서 채널 수를 줄이고 공간적 해상도를 복원
    - 최종적으로 입력 이미지와 동일한 크기의 출력 맵을 만듦 

- 스킵 커넥션
    - U-Net의 인코더의 각 단계에서 특징 맵을 추출하고 이 맵을 디코더의 대칭되는 단계로 직접 연결(concatenate)함
    - 인코더의 다운 샘플링 과정에서 손실될 수 있는 세부적인 지역 공간 정보를 디코더에 전달하여, 더 정확한 경계와 위치를 예측할 수 있도록 함

---

### Convoulution 에서의 Stride와 Padding

- Stride: 필터(kernel)가 입력 이미지 위를 이동하는 **간격**을 의미한다. 스트라이드가 2이면 필터가 두칸씩 이동하며, 이로 인해 출력 특징 맵의 크기는 줄어든다. 즉 image의 특징을 요약하는 것과 유사하다. 

- Padding: 입력 이미지의 가장자리의 0을 추가하여 필터가 가장자리 픽셀에 충분히 접근할 수 있도록 돕는다. 이를 통해 출력 특징 맵의 크기가 입력과 동일하거나 비슷하게 유지될 수 있다.

### Transpose Convolution 에서의 Stride와 Padding 

- Stride: 전치 컨볼루션에서 스트라이드는 입력 픽셀 사이에 삽입되는 0의 개수를 결정하는 역할을 한다. 스트라이드가 2라면, 입력 특징 맵의 각 픽셀 사이에 1줄(stride - 1)의 0이 삽입 되어 이미지의 크기를 늘린다. 따라서 Transpose Convolution에서의 stride는 이미지의 크기를 확장하는 업샘플링, 즉 디코더에서 사용된다.

- Padding: 출력 이미지의 경계에서 얼마나 많은 픽셀을 제거할지를 결정한다. 즉, 입력에 0을 채우는 것이 아니라 계산된 출력에서 일부분을 잘라내는 역할을 한다. 


### U-Net Architecture

In [2]:
IMG_SIZE = 16 # Due to stride and pooling, must be divisible by 2 multiple times
IMG_CH = 1 # Black and white image, no color channels
BATCH_SIZE = 128

### Down Block

Convolution layers를 통해 입력 이미지에서 지역 정보를 추출하는 다운 샘플링 방식이다. 

In [ ]:
import torch 
import torch.nn as nn

class DownBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        kernel_size = 3
        stride = 1
        padding = 1
        
        super().__init__()
        # Convolution Block 
        layers = [
            nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(),
            nn.Conv2d(out_ch, out_ch, kernel_size, stride, padding),
            nn.BatchNorm1d(out_ch),
            nn.ReLU(),
            nn.MaxPool2d(2)
        ]
        
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)

### UpBlock

In [3]:
class UpBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        # Convolution variables
        kernel_size = 3
        stride = 1
        padding = 1

        # Transpose variables
        strideT = 2 # 값과 값 사이에 0의 값 추가 
        out_paddingT = 1

        super().__init__()
        # 2 * in_chs for concatednated skip connection
        # Convolution Block의 구성을 유사하지만 stride와 padding의 역할이 다름 
        layers = [
            # Down의 마지막 feature map이 그대로 Up의 처음 feature map에 더해짐 -> 섬세한 복원 가능 
            # 따라서 in_channel의 값이 2배임 
            nn.ConvTranspose2d(2 * in_ch, out_ch, kernel_size, strideT, padding, out_paddingT),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(),
            nn.Conv2d(out_ch, out_ch, kernel_size, stride, padding),
            nn.BatchNorm2d(out_ch),
            nn.ReLU()
        ]
        self.model = nn.Sequential(*layers)
    
    # skip: encoder의 대칭되는 위치에서 넘어온 tensor 
    # torch.cat((x, skip))은 x와 skip 텐서를 채널을 기준으로 concate
    def forward(self, x, skip):
        x = torch.cat((x, skip), dim = 1)
        x = self.model(x)
        return x

### Full U-Net

In [4]:
class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        img_ch = IMG_CH
        down_chs = (16, 32, 64)
        up_chs = down_chs[::-1]  # Reverse of the down channels
        latent_image_size = IMG_SIZE // 4 # 2 ** (len(down_chs) - 1)

        # Inital convolution
        self.down0 = nn.Sequential(
            nn.Conv2d(img_ch, down_chs[0], 3, padding=1),
            nn.BatchNorm2d(down_chs[0]),
            nn.ReLU()
        )

        # Downsample
        self.down1 = DownBlock(down_chs[0], down_chs[1])
        self.down2 = DownBlock(down_chs[1], down_chs[2])
        self.to_vec = nn.Sequential(nn.Flatten(), nn.ReLU())
        
        # Embeddings
        self.dense_emb = nn.Sequential(
            nn.Linear(down_chs[2]*latent_image_size**2, down_chs[1]),
            nn.ReLU(),
            nn.Linear(down_chs[1], down_chs[1]),
            nn.ReLU(),
            nn.Linear(down_chs[1], down_chs[2]*latent_image_size**2),
            nn.ReLU()
        )
        
        # Upsample
        self.up0 = nn.Sequential(
            nn.Unflatten(1, (up_chs[0], latent_image_size, latent_image_size)),
            nn.Conv2d(up_chs[0], up_chs[0], 3, padding=1),
            nn.BatchNorm2d(up_chs[0]),
            nn.ReLU(),
        )
        self.up1 = UpBlock(up_chs[0], up_chs[1])
        self.up2 = UpBlock(up_chs[1], up_chs[2])

        # Match output channels
        self.out = nn.Sequential(
            nn.Conv2d(up_chs[-1], up_chs[-1], 3, 1, 1),
            nn.BatchNorm2d(up_chs[-1]),
            nn.ReLU(),
            nn.Conv2d(up_chs[-1], img_ch, 3, 1, 1),
        )

    def forward(self, x):
        down0 = self.down0(x)
        down1 = self.down1(down0)
        down2 = self.down2(down1)
        latent_vec = self.to_vec(down2)

        up0 = self.up0(latent_vec)
        up1 = self.up1(up0, down2)
        up2 = self.up2(up1, down1)
        return self.out(up2)

In [5]:
model = UNet()
print("Num params: ", sum(p.numel() for p in model.parameters()))

Num params:  234977
